# Task5 变形

## 1 知识梳理（重点记忆）

### 1.1 长宽表变形
- pivot函数，使用场景：长表变宽表
- pivot_table函数，使用场景：长表变宽表的同时，进行数据聚合  
参数说明：
|参数|描述|
|---|---|
|index|宽表的索引列|
|columns|宽表的列名|
|values|数据集，即资源|
|margins|bool类型，边际汇总|

- melt函数，使用场景：宽表变长表  
参数说明：
|参数|描述|
|---|---|
|id_vars|分组列|
|value_vars|长表中的类别|
|var_name|类别名|
|value_name|数据集，即资源|

- wide_to_long函数，使用场景：列名拆分，即列中包含交叉类别
参数说明：
|参数|描述|
|---|---|
|stubnames|分隔列名之后的前缀列名|
|i|索引列|
|j|分隔列名之后的后缀分组列名|
|sep|列名分隔符|
|suffix|正则表达式|

### 1.2 索引的变形
- stack函数，使用场景：把列索引的层压入行索引
- unstack函数，使用场景：把行索引转为列索引

### 1.3 其他变形函数
- crosstab函数，使用场景：在默认状态下，可以统计元素组合出现的频数
- explode函数，使用场景：对某一列的元素进行纵向的展开，其类型必须为`list`, `tuple`, `Series`, `np.ndarray`中的一种

## 2 练一练

In [1]:
import pandas as pd
import numpy as np

### 2.1 第1题
在上面的边际汇总例子中，行或列的汇总为新表中行元素或者列元素的平均值，而总体的汇总为新表中四个元素的平均值。这种关系一定成立吗？若不成立，请给出一个例子来说明。

**我的解答：**

In [2]:
df = pd.DataFrame({'Name':['San Zhang', 'San Zhang', 
                              'San Zhang', 'San Zhang',
                              'Si Li', 'Si Li', 'Si Li', 'Si Li'],
                   'Subject':['Chinese', 'Chinese', 'Math', 'Math',
                                 'Chinese', 'Chinese', 'Math', 'Math'],
                   'Grade':[80, 90, 100, 90, 70, 80, 85, 95]})
df

,Name,Subject,Grade
0,San Zhang,Chinese,80
1,San Zhang,Chinese,90
2,San Zhang,Math,100
3,San Zhang,Math,90
4,Si Li,Chinese,70
5,Si Li,Chinese,80
6,Si Li,Math,85
7,Si Li,Math,95


In [3]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc=lambda x:x.min(),
               margins=True)

Subject,Chinese,Math,All
Name,,,
San Zhang,80,90,80
Si Li,70,85,70
All,70,85,70


上面的说法不正确，总体的汇总为应该为所有Grade的值进行聚合，而不是新表中四个元素，通过查看pandas源码，`pivot.py`下的`_compute_grand_margin`函数，描述了整个计算过程：  
- 遍历data[values]
- 根据给定的函数，计算grand_margin[k] = aggfunc(v)
- 最后返回计算得到的grand_margin

```python
def _compute_grand_margin(data, values, aggfunc, margins_name: str = "All"):
    if values:
        grand_margin = {}
        for k, v in data[values].items():
            try:
                if isinstance(aggfunc, str):
                    grand_margin[k] = getattr(v, aggfunc)()
                elif isinstance(aggfunc, dict):
                    if isinstance(aggfunc[k], str):
                        grand_margin[k] = getattr(v, aggfunc[k])()
                    else:
                        grand_margin[k] = aggfunc[k](v)
                else:
                    grand_margin[k] = aggfunc(v)
            except TypeError:
                pass
        return grand_margin
    else:
        return {margins_name: aggfunc(data.index)}
```

### 2.2 第2题
前面提到了`crosstab`的性能劣于`pivot_table`，请选用多个聚合方法进行验证。

**我的解答：**

In [4]:
df = pd.read_csv('../data/learn_pandas.csv')
df.head()

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
0,Shanghai Jiao Tong University,Freshman,Gaopeng Yang,Female,158.9,46.0,N,1,2019/10/5,0:04:34
1,Peking University,Freshman,Changqiang You,Male,166.5,70.0,N,1,2019/9/4,0:04:20
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,188.9,89.0,N,2,2019/9/12,0:05:22
3,Fudan University,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N,2,2020/1/3,0:04:08
4,Fudan University,Sophomore,Gaojuan You,Male,174.0,74.0,N,2,2019/11/6,0:05:22


1. `mean`聚合函数，统计身高`Height`的均值

In [5]:
%timeit -n 30 pd.crosstab(index = df.School, columns = df.Transfer, values = df.Height, aggfunc = 'mean')

30 loops, best of 5: 9.62 ms per loop


In [6]:
%timeit -n 30 df.pivot_table(index = 'School', columns = 'Transfer', values = 'Height', aggfunc = 'mean')

30 loops, best of 5: 9.08 ms per loop


2. `max`聚合函数，统计体重`Weight`的最大值

In [7]:
%timeit -n 30 pd.crosstab(index = df.School, columns = df.Transfer, values = df.Weight, aggfunc = 'max')

30 loops, best of 5: 11.1 ms per loop


In [8]:
%timeit -n 30 df.pivot_table(index = 'School', columns = 'Transfer', values = 'Weight', aggfunc = 'max')

30 loops, best of 5: 10.2 ms per loop


## 3 练习
### 3.1 Ex1：美国非法药物数据集

现有一份关于美国非法药物的数据集，其中`SubstanceName, DrugReports`分别指药物名称和报告数量：

In [9]:
df = pd.read_csv('../data/drugs.csv').sort_values(['State','COUNTY','SubstanceName'],ignore_index=True)
df.head(3)

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2011,KY,ADAIR,Buprenorphine,3
1,2012,KY,ADAIR,Buprenorphine,5
2,2013,KY,ADAIR,Buprenorphine,4


1. 将数据转为如下的形式：

<img src="../source/_static/Ex5_1.png" width="35%">

2. 将第1问中的结果恢复为原表。
3. 按`State`分别统计每年的报告数量总和，其中`State, YYYY`分别为列索引和行索引，要求分别使用`pivot_table`函数与`groupby+unstack`两种不同的策略实现，并体会它们之间的联系。

**我的解答：**  

**第1问：**

In [10]:
df_pivot = df.pivot(index=['State', 'COUNTY', 'SubstanceName'], columns='YYYY', values='DrugReports')
df_pivot = df_pivot.reset_index().rename_axis(columns={'YYYY':''})
df_pivot.head()

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,NaN,3.0,5.0,4.0,27.0,5.0,7.0,10.0
1,KY,ADAIR,Codeine,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
2,KY,ADAIR,Fentanyl,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,KY,ADAIR,Heroin,NaN,NaN,1.0,2.0,NaN,1.0,NaN,2.0
4,KY,ADAIR,Hydrocodone,6.0,9.0,10.0,10.0,9.0,7.0,11.0,3.0


**第2问：** 使用`melt`进行恢复

In [11]:
df_melt = df_pivot.melt(id_vars=['State', 'COUNTY', 'SubstanceName'],
              value_vars=df_pivot.columns[3:],
              var_name='YYYY',
              value_name='DrugReports')
df_melt.head(3)

,State,COUNTY,SubstanceName,YYYY,DrugReports
0,KY,ADAIR,Buprenorphine,2010,NaN
1,KY,ADAIR,Codeine,2010,NaN
2,KY,ADAIR,Fentanyl,2010,NaN


In [12]:
df.head(3)

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2011,KY,ADAIR,Buprenorphine,3
1,2012,KY,ADAIR,Buprenorphine,5
2,2013,KY,ADAIR,Buprenorphine,4


In [13]:
df_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49712 entries, 0 to 49711
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   State          49712 non-null  object 
 1   COUNTY         49712 non-null  object 
 2   SubstanceName  49712 non-null  object 
 3   YYYY           49712 non-null  object 
 4   DrugReports    24062 non-null  float64
dtypes: float64(1), object(4)
memory usage: 1.9+ MB


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24062 entries, 0 to 24061
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   YYYY           24062 non-null  int64 
 1   State          24062 non-null  object
 2   COUNTY         24062 non-null  object
 3   SubstanceName  24062 non-null  object
 4   DrugReports    24062 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 940.0+ KB


观察可知，`df_melt`需要再进行如下操作，才能和`df`一致：
- 将`DrugReports`列为`NaN`的行删除
- 重新将列进行重排，然后将`YYYY`的类型修改为`int64`，将`DrugReports`的类型修改为`int64`

In [15]:
df_melt.dropna(subset=['DrugReports'], inplace=True)
df_melt.head()

,State,COUNTY,SubstanceName,YYYY,DrugReports
4,KY,ADAIR,Hydrocodone,2010,6.0
6,KY,ADAIR,Methadone,2010,1.0
13,KY,ALLEN,Hydrocodone,2010,10.0
15,KY,ALLEN,Methadone,2010,4.0
17,KY,ALLEN,Oxycodone,2010,15.0


In [16]:
df_melt = df_melt[df.columns].sort_values(['State', 'COUNTY', 'SubstanceName'], ignore_index=True)
df_melt = df_melt.astype({'YYYY':'int64', 'DrugReports':'int64'})

In [17]:
df_melt.equals(df)

True

**第3问：** 按State分别统计每年的报告数量总和，其中State, YYYY分别为列索引和行索引，要求分别使用pivot_table函数与groupby+unstack两种不同的策略实现，并体会它们之间的联系。  
1. 使用`pivot_table`函数

In [18]:
df.pivot_table(index='YYYY', columns='State', values='DrugReports', aggfunc='sum')

State,KY,OH,PA,VA,WV
YYYY,,,,,
2010,10453,19707,19814,8685,2890
2011,10289,20330,19987,6749,3271
2012,10722,23145,19959,7831,3376
2013,11148,26846,20409,11675,4046
2014,11081,30860,24904,9037,3280
2015,9865,37127,25651,8810,2571
2016,9093,42470,26164,10195,2548
2017,9394,46104,27894,10448,1614


2. 使用`groupby`和`unstack`方法

In [19]:
df_ex1_3 = df.groupby(['State', 'YYYY'])['DrugReports'].sum().to_frame()
df_ex1_3.head(5)

DrugReports
State YYYY             
KY    2010        10453
      2011        10289
      2012        10722
      2013        11148
      2014        11081

In [20]:
df_ex1_3 = df_ex1_3.unstack(0)
df_ex1_3

DrugReports                           
State          KY     OH     PA     VA    WV
YYYY                                        
2010        10453  19707  19814   8685  2890
2011        10289  20330  19987   6749  3271
2012        10722  23145  19959   7831  3376
2013        11148  26846  20409  11675  4046
2014        11081  30860  24904   9037  3280
2015         9865  37127  25651   8810  2571
2016         9093  42470  26164  10195  2548
2017         9394  46104  27894  10448  1614

In [21]:
# 删掉DrugReports，使用droplevel方法
df_ex1_3.droplevel(level=0, axis=1)

State,KY,OH,PA,VA,WV
YYYY,,,,,
2010,10453,19707,19814,8685,2890
2011,10289,20330,19987,6749,3271
2012,10722,23145,19959,7831,3376
2013,11148,26846,20409,11675,4046
2014,11081,30860,24904,9037,3280
2015,9865,37127,25651,8810,2571
2016,9093,42470,26164,10195,2548
2017,9394,46104,27894,10448,1614


### 3.2 Ex2：特殊的wide_to_long方法

从功能上看，`melt`方法应当属于`wide_to_long`的一种特殊情况，即`stubnames`只有一类。请使用`wide_to_long`生成`melt`一节中的`df_melted`。（提示：对列名增加适当的前缀）

In [22]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang', 'Si Li'],
                   'Chinese':[80, 90],
                   'Math':[80, 75]})
df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [23]:
df_melted = df.melt(id_vars = ['Class', 'Name'],
                    value_vars = ['Chinese', 'Math'],
                    var_name = 'Subject',
                    value_name = 'Grade')
df_melted

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,2,Si Li,Chinese,90
2,1,San Zhang,Math,80
3,2,Si Li,Math,75


**我的解答：**

In [24]:
# 对列名增加适当的前缀
df_ex2 = df.copy()
df_ex2.rename(columns={'Chinese':'my_Chinese', 'Math':'my_Math'}, inplace=True)
df_ex2.head()

,Class,Name,my_Chinese,my_Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [25]:
df_ex2 = pd.wide_to_long(df_ex2,
                stubnames=['my'],
                i = ['Class', 'Name'],
                j='Subject',
                sep='_',
                suffix='.+')
df_ex2 = df_ex2.reset_index()
df_ex2

,Class,Name,Subject,my
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,80
2,2,Si Li,Chinese,90
3,2,Si Li,Math,75


In [26]:
# 修改列名
df_ex2.rename(columns={'my':'Grade'}, inplace=True)

In [27]:
# 按照Subject排序并忽略index列
df_ex2.sort_values(['Subject'], inplace=True, ignore_index=True)
df_ex2

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,2,Si Li,Chinese,90
2,1,San Zhang,Math,80
3,2,Si Li,Math,75


In [28]:
df_ex2.equals(df_melted)

True